ctypes è un'alternativa a pybind e a cython

devo scrivere la shared library e compilarla manualmente (usando come sempre 'g++ -fPIC -shared -o libblabla.so blabla.cpp') come se avessi intenzione di chiamarla da un codice c/c++; l'unica differenza è che se scrivo la libreria in c++ devo aggiungere la parola chiave 'extern "C"{...}' (perché ctype sarebbe fatto per leggere codice c)

nel codice python devo importare il package e creare una variabile contenente il path della libreria; poi posso chiamare direttamente le funzioni della libreria usando la variabile definita

In [4]:
import ctypes
 
libObject = ctypes.CDLL('./libhello.so')

In [5]:
# notare l'output numerico indesiderato (di provenienza ignota)

libObject.hello()

hello world!


1492976960

In [6]:
libObject.sum(1, 2)

3

la funzione sum() deve indovinare il tipo, pertanto a volte potrebbe non funzionare: meglio passare anche il tipo, come nella cella sotto

In [8]:
libObject.sum.argtypes = [ctypes.c_int, ctypes.c_int]
libObject.sum.restype = ctypes.c_int

libObject.sum(1, 2)

3

#### fare tutto dal notebook
posso farlo usando le funzioni magiche (l'unico problema è che non ho la syntax highlight di c/c++)

In [1]:
%%file hello2.cpp
#include <iostream>


extern "C"{

    const int sum(const int a, const int b) {
        return a + b;
    }

}

Overwriting hello2.cpp


In [2]:
%%bash
g++ -fPIC -shared -o libhello2.so hello2.cpp

In [5]:
libObject2 = ctypes.CDLL('./libhello2.so')

libObject2.sum.argtypes = [ctypes.c_int, ctypes.c_int]
libObject2.sum.restype = ctypes.c_int

libObject2.sum(1, 2)

3

#### speed
come si vede dalle celle sotto non si guadagna niente (anzi si perde) in performance riscrivendo semplicemente la stessa identica funzione in c++; il miglioramento si vede quando si inizia a sfruttare al massimo le caratteristiche di c++

In [17]:
def sum(a, b):
    return a + b

In [20]:
%%timeit
sum(1, 2)

117 ns ± 3.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [21]:
libObject2.sum.argtypes = [ctypes.c_int, ctypes.c_int]
libObject2.sum.restype = ctypes.c_int

In [22]:
%%timeit
libObject2.sum(1, 2)

672 ns ± 14.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
